# Phi-3.5-mini-UA-spectrum fine-tuning notebook


## Setup

In [1]:
! pip install datasets transformers trl accelerate scipy
! pip install ninja packaging
! MAX_JOBS=6 pip install flash-attn --no-build-isolation --upgrade
! pip install wandb

In [2]:
from google.colab import userdata
import os

os.environ["HF_HUB_TOKEN"] = userdata.get('HF_TOKEN')
os.environ["WANDB_API_KEY"] = userdata.get('WANDB_API_KEY')

In [3]:
from huggingface_hub import login
import os

login(token=os.getenv("HF_HUB_TOKEN"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Data preparation

The datasets used have different formats.
We prepare and mix them in a single dataset.

In [4]:
from datasets import load_dataset, Dataset, concatenate_datasets, Features, Value
from transformers import AutoTokenizer

# Define all columns as strings
features = Features({
    'input': Value('string'),
    'output': Value('string'),
    'instruct': Value('string'),
    'dataset_type': Value('string'),
    'dataloader_name': Value('string')
})

In [5]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct", trust_remote_code=True)
tokenizer.padding_side = 'right'

dataset = load_dataset("ostapbodnar/ua-gec-pos-ner-golden", features=features)

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruct', 'dataset_type', 'dataloader_name'],
        num_rows: 213960
    })
    validation: Dataset({
        features: ['input', 'output', 'instruct', 'dataset_type', 'dataloader_name'],
        num_rows: 53490
    })
    test: Dataset({
        features: ['input', 'output', 'instruct', 'dataset_type', 'dataloader_name'],
        num_rows: 78990
    })
})

In [7]:
len(dataset["test"])

78990

In [8]:
from datasets import DatasetDict

sampled_dataset = DatasetDict({
    "train": dataset["train"].shuffle(seed=42).select(range(30000)),
    "test": dataset["test"].shuffle(seed=42).select(range(1500)),
})
sampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruct', 'dataset_type', 'dataloader_name'],
        num_rows: 30000
    })
    test: Dataset({
        features: ['input', 'output', 'instruct', 'dataset_type', 'dataloader_name'],
        num_rows: 1500
    })
})

In [9]:
def create_message_column(row):
    messages = []
    user = {
        "content": f"{row['instruct']}\n Input: {row['input']}",
        "role": "user"
    }
    messages.append(user)
    assistant = {
        "content": f"{row['output']}",
        "role": "assistant"
    }
    messages.append(assistant)
    return {"messages": messages}

def format_dataset_chatml(row):
    return {"text": tokenizer.apply_chat_template(row["messages"], add_generation_prompt=False, tokenize=False)}

In [10]:
dataset_chatml = sampled_dataset.map(create_message_column)
dataset_chatml = dataset_chatml.map(format_dataset_chatml)

In [11]:
dataset_chatml

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruct', 'dataset_type', 'dataloader_name', 'messages', 'text'],
        num_rows: 30000
    })
    test: Dataset({
        features: ['input', 'output', 'instruct', 'dataset_type', 'dataloader_name', 'messages', 'text'],
        num_rows: 1500
    })
})

In [12]:
print(dataset_chatml["train"][587]["text"])

<|user|>
Ідентифікуй жанр новини на основі тексту.
 Input: Заголовок: {Шахтар} – {Сілекс} ⇒ Дивитися онлайн текстову трансляцію ≺{26.01.2021}≻ {Футбол} на СПОРТ.UA, текст: У вівторок, 26-го січня, відбудеться товариський поєдинок, в якому донецький «Шахтар» зіграє з македонським «Сілексом». Матч пройде в Анталії, початок гри о 10:00. На турецькому зборі діючі чемпіони України провели вже два спаринги: з польським «Лехом» (1:1) і болгарським «Лудогорцем» (2:2). Матч проти «срібного» призера минулого розіграшу чемпіонату Словенії - «Марібора», був скасований через спалах коронавірусу у словенців. «Сілекс» в Туреччині також без перемог. Команда на зборі провела два поєдинки і в обох зазнала поразки. В одному з них - від ковалівського «Колоса» Sport.ua проведе текстову трансляцію матчу «Шахтар» - «Сілекс». За перебігом поєдинку можна слідкувати за цим посиланням.<|end|>
<|assistant|>
спорт<|end|>
<|endoftext|>


In [13]:
print(dataset_chatml["train"][587]["text"])

<|user|>
Ідентифікуй жанр новини на основі тексту.
 Input: Заголовок: {Шахтар} – {Сілекс} ⇒ Дивитися онлайн текстову трансляцію ≺{26.01.2021}≻ {Футбол} на СПОРТ.UA, текст: У вівторок, 26-го січня, відбудеться товариський поєдинок, в якому донецький «Шахтар» зіграє з македонським «Сілексом». Матч пройде в Анталії, початок гри о 10:00. На турецькому зборі діючі чемпіони України провели вже два спаринги: з польським «Лехом» (1:1) і болгарським «Лудогорцем» (2:2). Матч проти «срібного» призера минулого розіграшу чемпіонату Словенії - «Марібора», був скасований через спалах коронавірусу у словенців. «Сілекс» в Туреччині також без перемог. Команда на зборі провела два поєдинки і в обох зазнала поразки. В одному з них - від ковалівського «Колоса» Sport.ua проведе текстову трансляцію матчу «Шахтар» - «Сілекс». За перебігом поєдинку можна слідкувати за цим посиланням.<|end|>
<|assistant|>
спорт<|end|>
<|endoftext|>


We can then check how many examples will be truncated if we choose a maximum length of X tokens (2048 in this case).

In [14]:
from scipy.stats import percentileofscore
import multiprocessing

def calculate_lengths(batch):
    return {"conv_lengths": [len(tokenizer(text)["input_ids"]) for text in batch["text"]]}

conv_lengths = dataset_chatml["train"].map(
    calculate_lengths,
    batched=True,
    batch_size=1000,
    num_proc=multiprocessing.cpu_count()
)["conv_lengths"]

In [15]:
chosen_length=2048

percentile = percentileofscore(conv_lengths, chosen_length)
print(percentile)

96.24666666666667


## Load model

For Spectrum, we need to load the model using Transformers, no quantization.

In [16]:
from transformers import AutoModelForCausalLM
import torch

model_id = "microsoft/Phi-3.5-mini-instruct"


model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct",
    use_cache=False,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="auto",
    trust_remote_code=True
)
model = model.half()

# reference: https://huggingface.co/microsoft/Phi-3.5-mini-instruct/blob/main/sample_finetune.py
# keep in mind that setting tokenizer.model_max_length = 2048 as suggested is WRONG
tokenizer.pad_token = tokenizer.unk_token  # use unk rather than eos token to prevent endless generation
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
tokenizer.padding_side = 'right'

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Apply Spectrum
https://github.com/cognitivecomputations/spectrum
https://arxiv.org/abs/2406.06623

In short, when using Spectrum, we only fine-tune some layers of the model with high Signal-to-Noise Ratio.
So, we need to freeze the other layers before training.

---

I computed the following YAML file using the Spectrum script, which unfortunately is not compatible with notebook environments.

```bash
# installation
git clone https://github.com/cognitivecomputations/spectrum.git
cd spectrum
pip install -r requirements.txt

# run
python spectrum.py --model-name microsoft/Phi-3.5-mini-instruct --top-percent 30
```

This command first scans the model (if not available) and then produces the YAML file with top SNR layers.

In [17]:
# For simplicity, I'm pasting the YAML parameters here

yaml_parameters="""unfrozen_parameters:
- ^lm_head.weight$
- ^model.embed_tokens.weight$
# mlp.down_proj layers
- model.layers.2.mlp.down_proj
- model.layers.3.mlp.down_proj
- model.layers.1.mlp.down_proj
- model.layers.23.mlp.down_proj
- model.layers.4.mlp.down_proj
- model.layers.26.mlp.down_proj
- model.layers.25.mlp.down_proj
- model.layers.24.mlp.down_proj
- model.layers.28.mlp.down_proj
# mlp.gate_up_proj layers
- model.layers.31.mlp.gate_up_proj
- model.layers.4.mlp.gate_up_proj
- model.layers.3.mlp.gate_up_proj
- model.layers.5.mlp.gate_up_proj
- model.layers.6.mlp.gate_up_proj
- model.layers.2.mlp.gate_up_proj
- model.layers.30.mlp.gate_up_proj
- model.layers.9.mlp.gate_up_proj
- model.layers.28.mlp.gate_up_proj
# self_attn.o_proj layers
- model.layers.0.self_attn.o_proj
- model.layers.1.self_attn.o_proj
- model.layers.10.self_attn.o_proj
- model.layers.11.self_attn.o_proj
- model.layers.9.self_attn.o_proj
- model.layers.3.self_attn.o_proj
- model.layers.19.self_attn.o_proj
- model.layers.8.self_attn.o_proj
- model.layers.4.self_attn.o_proj
# self_attn.qkv_proj layers
- model.layers.23.self_attn.qkv_proj
- model.layers.24.self_attn.qkv_proj
- model.layers.22.self_attn.qkv_proj
- model.layers.26.self_attn.qkv_proj
- model.layers.27.self_attn.qkv_proj
- model.layers.25.self_attn.qkv_proj
- model.layers.28.self_attn.qkv_proj
- model.layers.29.self_attn.qkv_proj
- model.layers.31.self_attn.qkv_proj
"""

In [18]:
unfrozen_parameters = []
for line in yaml_parameters.splitlines():
  if line.startswith("- "):
    unfrozen_parameters.append(line.split("- ")[1])

In [19]:
import re

def _freeze_and_unfreeze_parameters(model, unfrozen_parameters):
    # freeze all parameters
    for param in model.parameters():
        param.requires_grad = False
    # unfreeze Spectrum parameters
    for name, param in model.named_parameters():
        if any(re.match(unfrozen_param, name) for unfrozen_param in unfrozen_parameters):
            param.requires_grad = True

In [20]:
_freeze_and_unfreeze_parameters(model, unfrozen_parameters)

In [21]:
# check the outcome of our freezing operation
for name, param in model.named_parameters():
    if param.requires_grad:
      print(name, param.requires_grad)

# model.embed_tokens.weight True
# model.layers.0.self_attn.o_proj.weight True
# model.layers.1.self_attn.o_proj.weight True
# model.layers.1.mlp.down_proj.weight True
# ...

model.embed_tokens.weight True
model.layers.0.self_attn.o_proj.weight True
model.layers.1.self_attn.o_proj.weight True
model.layers.1.mlp.down_proj.weight True
model.layers.2.mlp.gate_up_proj.weight True
model.layers.2.mlp.down_proj.weight True
model.layers.3.self_attn.o_proj.weight True
model.layers.3.mlp.gate_up_proj.weight True
model.layers.3.mlp.down_proj.weight True
model.layers.4.self_attn.o_proj.weight True
model.layers.4.mlp.gate_up_proj.weight True
model.layers.4.mlp.down_proj.weight True
model.layers.5.mlp.gate_up_proj.weight True
model.layers.6.mlp.gate_up_proj.weight True
model.layers.8.self_attn.o_proj.weight True
model.layers.9.self_attn.o_proj.weight True
model.layers.9.mlp.gate_up_proj.weight True
model.layers.10.self_attn.o_proj.weight True
model.layers.11.self_attn.o_proj.weight True
model.layers.19.self_attn.o_proj.weight True
model.layers.22.self_attn.qkv_proj.weight True
model.layers.23.self_attn.qkv_proj.weight True
model.layers.23.mlp.down_proj.weight True
model.

## Training configuration

In [22]:
# WANDB configuration (optional)

import wandb
import os

os.environ["PROJECT"]="phi3.5-mini-ua-golden"

project_name = os.environ["PROJECT"]

wandb.init(project=project_name, name = project_name)

wandb: Currently logged in as: ostapbodnar (ostap-bodnar). Use `wandb login --relogin` to force relogin


In [23]:
from trl import SFTConfig, SFTTrainer

new_model_id="ostapbodnar/Phi3.5-mini-instruct-UA-spectrum"

cfg = SFTConfig(
    output_dir='./mymodel',
    overwrite_output_dir=True,
    hub_model_id=new_model_id,
    hub_strategy="every_save",
    save_strategy="steps",
    save_steps=500,
    save_total_limit=1,
    push_to_hub=True,
    do_eval=True,
    evaluation_strategy="steps",
    logging_steps=100,
    max_seq_length=3076,                    # reduced sequence length for lower memory usage
    dataset_text_field="text",              # since we already prepared the dataset, let's point the Trainer to the correct column
    remove_unused_columns=True,
    packing=True,                           # speeds up training. https://huggingface.co/docs/trl/en/sft_trainer#packing-dataset--constantlengthdataset-
    num_train_epochs=2,
    lr_scheduler_type="cosine",
    warmup_ratio=0.2,
    bf16=True,                             # switched to fp16 for reduced memory consumption
    tf32=False,                             # disabled tf32 to reduce memory usage
    learning_rate=5.0e-06,                  # suggested in https://huggingface.co/microsoft/Phi-3.5-mini-instruct/blob/main/sample_finetune.py
    per_device_train_batch_size=3,          # reduced batch size to avoid memory error
    gradient_accumulation_steps=2,          # reduced gradient accumulation to lower memory consumption
    per_device_eval_batch_size=3,
    eval_steps=1000,
    gradient_checkpointing=True             # enabled gradient checkpointing to save memory
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
sft_trainer = SFTTrainer(
    model=model,
    args=cfg,
    train_dataset=dataset_chatml["train"],
    eval_dataset=dataset_chatml['test'],
    tokenizer=tokenizer
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [25]:
sft_trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[at

Step,Training Loss,Validation Loss
1000,0.000000,nan
2000,0.000000,nan


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an except

TrainOutput(global_step=2062, training_loss=3.0449157182044133, metrics={'train_runtime': 7557.9715, 'train_samples_per_second': 1.637, 'train_steps_per_second': 0.273, 'total_flos': 8.499361570890301e+17, 'train_loss': 3.0449157182044133, 'epoch': 1.9990305380513815})

In [26]:
sft_trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/ostapbodnar/Phi3.5-mini-instruct-UA-spectrum/commit/dfcd5d7cafe9f80a657a810b2988d685b6dd70c2', commit_message='End of training', commit_description='', oid='dfcd5d7cafe9f80a657a810b2988d685b6dd70c2', pr_url=None, pr_revision=None, pr_num=None)

In [27]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.eos_token)
tokenizer.padding_side = 'left'

tokenizer.push_to_hub(new_model_id)

README.md:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ostapbodnar/Phi3.5-mini-instruct-UA-spectrum/commit/45469953b15c96a45597c2435f9efc0f0e20ed0d', commit_message='Upload tokenizer', commit_description='', oid='45469953b15c96a45597c2435f9efc0f0e20ed0d', pr_url=None, pr_revision=None, pr_num=None)

I finally did some manual updates on the model repo:
- copying some files from the original model to my model...
- modifying config.json and generation_config.json to use the right tokens ids for `eos_token_id`.